# Installation and Setup

You must reinstall everything each time you restart the machine. If already downloaded, dependencies will be auto-updated.

In [ ]:
symlink_to_notebooks = False
model_storage_dir = '/storage/models'
repo_storage_dir = '/notebooks'

activate_xformers = False                              # Enables the xformers optimizations using pre-built wheels.
                                                       # Setting to True will automatically set up your environment/machine for xformers. 
link_novelai_anime_vae = False                         # Enables the linking of animevae.pt to each of the NovelAI models.
                                                       # Set to True if you've downloaded both the NovelAI models and hypernetworks.
download_scripts = True                               # Download custom scripts? Only reason why you would leave it disabled is because it may
                                                       # take a while to complete.
activate_deepdanbooru = False                          # Enable and install DeepDanbooru -> https://github.com/KichangKim/DeepDanbooru
activate_medvram = False                                # Enable medvram option.
                                                       # These are model optimizations which will reduce VRAM usage at the expense of some speed.
                                                       # Set to False if you have a lot of VRAM.
disable_pickle_check = True                           # Disable the automatic check for unexpected data in model files.
                                                       # Leave this set to False unless you have a reason to disable the check.
gradio_port = False                                    # Launch Gradio on a specific port. Set to False to let Gradio choose a port.
                                                       # This disables online Gradio app mode and you will only be able to access it on your local network.
gradio_auth = False                                    # Enable gradio_auth and insecure-extension-access option.
                                                       # Set to "me:password" to let Gradio auth.

# ===============================================================
# Save variables to Jupiter's temp storage so we can access it even if the kernel restarts.
%store symlink_to_notebooks model_storage_dir repo_storage_dir activate_xformers link_novelai_anime_vae download_scripts activate_deepdanbooru activate_medvram disable_pickle_check gradio_port gradio_auth

## Clone the central repository

In [ ]:
import os
%store -r symlink_to_notebooks model_storage_dir repo_storage_dir
%cd /notebooks/

def delete_broken_symlinks(path):
    # make sure to pass this function a path without a trailing slash
    for file in os.listdir(path):
        if os.path.islink(f'{path}/{file}') and not os.path.exists(os.readlink(f'{path}/{file}')):
            print(f'Symlink broken, removing: {file}')
            os.unlink(f'{path}/{file}')

def update_repo_if_not_exists(path, repo_clone_url, pre=None):
    if pre is not None:
        pre()    
    if not os.path.exists(path):
        !git clone "{repo_clone_url}" "{path}"
    else:
        print(f'{repo_clone_url.split("/")[-1]} already downloaded, updating...')
        !cd "{path}" && git pull # no % so we don't interfere with the main process

def init_free():
    if (symlink_to_notebooks and repo_storage_dir != '/notebooks'):
        delete_broken_symlinks('/notebooks/') # remove broken symlinks since it might have been installed in a non-persistent directory
        if not os.path.exists(repo_storage_dir):
            !mkdir -p "{repo_storage_dir}"
            !ln -s "{repo_storage_dir}" /notebooks/
            !ls -la /notebooks/stable-diffusion
update_repo_if_not_exists(f'{repo_storage_dir}/stable-diffusion-webui', 'https://github.com/AUTOMATIC1111/stable-diffusion-webui', init_free)

## Install requirements and download repositories

In [ ]:
%store -r symlink_to_notebooks model_storage_dir repo_storage_dir activate_xformers download_scripts activate_deepdanbooru
%cd "{repo_storage_dir}/stable-diffusion-webui"
import os

# Import launch.py which will automatically run the install script but not launch the WebUI.
# They require a few specific external git repo commits so we have to do it their way. 
import launch
launch.prepare_enviroment()

# The installer isn't installing deepdanbooru right now so we'll do it manually
if activate_deepdanbooru:
    !pip install "git+https://github.com/KichangKim/DeepDanbooru.git@edf73df4cdaeea2cf00e9ac08bd8a9026b7a7b26#egg=deepdanbooru[tensorflow]" # tensorflow==2.10.0 tensorflow-io==0.27.0 flatbuffers==1.12

# latent-diffusion is a requirement but launch.py isn't downloading it so we'll do it manually.
if not os.path.exists(f'{repo_storage_dir}/stable-diffusion-webui/repositories/latent-diffusion'):
    !git clone https://github.com/crowsonkb/k-diffusion.git "{repo_storage_dir}/stable-diffusion-webui/repositories/k-diffusion"
    !git clone https://github.com/Hafiidz/latent-diffusion.git "{repo_storage_dir}/stable-diffusion-webui/repositories/latent-diffusion"

# Download popular custom scripts. This is basically remote code execution so be careful.
# See https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Custom-Scripts
if download_scripts:
    !pip install requests
    import shutil
    import requests
    !pip install moviepy==1.0.3
    !apt update
    !apt install -y potrace python3-tk

    def download_file_dir(url, output_dir):
        # output_dir must have a trailing slash
        local_filename = url.split('/')[-1]
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with open(f'{output_dir}{local_filename}', 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        return local_filename
    def do_script_download(scripts_list, domain, path):
        for item in scripts_list:
            download_file_dir(f'https://{domain}/{item}', path)
            print(f'{item.split("/")[-1]} downloaded...')

    do_script_download([
        'GRMrGecko/stable-diffusion-webui-automatic/advanced_matrix/scripts/advanced_prompt_matrix.py',
        'dfaker/stable-diffusion-webui-cv2-external-masking-script/main/external_masking.py',
        'memes-forever/Stable-diffusion-webui-video/main/videos.py',
        'yownas/seed_travel/main/scripts/seed_travel.py',
        'Animator-Anon/Animator/main/animation.py',
        'Filarius/stable-diffusion-webui/master/scripts/vid2vid.py',
        'GeorgLegato/Txt2Vectorgraphics/main/txt2vectorgfx.py',
        'yownas/shift-attention/main/scripts/shift_attention.py',
        'DiceOwl/StableDiffusionStuff/main/loopback_superimpose.py',
        'Engineer-of-Stuff/stable-diffusion-paperspace/main/lfs/save_steps.py',
        'Pfaeff/sd-web-ui-scripts/main/moisaic.py'
    ], 'raw.githubusercontent.com', f'{repo_storage_dir}/stable-diffusion-webui/scripts/')

    do_script_download([
        'dfaker/f88aa62e3a14b559fe4e5f6b345db664/raw/791dabfa0ab26399aa2635bcbc1cf6267aa4ffc2/alternate_sampler_noise_schedules.py',
        'camenduru/9ec5f8141db9902e375967e93250860f/raw/c1a03eb447548adbef1858c0e69d3567a390d2f4/run_n_times.py'
    ], 'gist.githubusercontent.com', f'{repo_storage_dir}/stable-diffusion-webui/scripts/')

    # Download and set up txt2img2img
    update_repo_if_not_exists(f'{repo_storage_dir}/stable-diffusion-webui/txt2img2img_root', 'https://github.com/ThereforeGames/txt2img2img.git')
    !cp -r "{repo_storage_dir}/stable-diffusion-webui/txt2img2img_root/scripts" "{repo_storage_dir}/stable-diffusion-webui"
    !cp -r "{repo_storage_dir}/stable-diffusion-webui/txt2img2img_root/txt2img2img" "{repo_storage_dir}/stable-diffusion-webui"
    !cp -r "{repo_storage_dir}/stable-diffusion-webui/txt2img2img_root/venv" "{repo_storage_dir}/stable-diffusion-webui"

    # Download and set up txt2mask
    update_repo_if_not_exists(f'{repo_storage_dir}/stable-diffusion-webui/txt2mask', 'https://github.com/ThereforeGames/txt2mask.git')
    !echo "Copying txt2mask..."
    !cp -r "{repo_storage_dir}/stable-diffusion-webui/txt2mask/repositories/clipseg" "{repo_storage_dir}/stable-diffusion-webui/repositories"
    !cp -r "{repo_storage_dir}/stable-diffusion-webui/txt2mask/scripts/" "{repo_storage_dir}/stable-diffusion-webui/"

    # Install the dynamic-prompts/wildcard script
    !git clone https://github.com/adieyal/sd-dynamic-prompting/ extensions/dynamic-prompts

if activate_xformers:
    print('Installing xformers...')
    import subprocess
    def download_release(url):
        binary = 'xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl' # have to save the binary as a specific name that pip likes
        tmp_dir = subprocess.check_output(['mktemp', '-d']).decode('ascii').strip('\n')
        !wget "{url}" -O "{tmp_dir}/{binary}"
        return os.path.join(tmp_dir, binary)

    # Set up pip packages
    !pip uninstall -y torch  torchvision torchaudio # Remove existing pytorch install.
    !pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113 # Install pytorch for cuda 11.3
    s = subprocess.getoutput('nvidia-smi')
    if 'A4000' in s:
        xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/A4000-Oct-28-2022/a4000-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
    elif 'A5000' in s:
        xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/A5000-Nov-1-2022/a5000-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
    elif 'A6000' in s:
        xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/A6000-Nov-1-2022/a6000-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
    elif 'P5000' in s:
        xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/P5000-Nov-1-2022/p5000-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
    elif 'RTX 4000' in s:
        xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/RTX-4000-Nov-1-2022/rtx4000-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
    elif 'RTX 5000' in s:
        xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/RTX-5000-Nov-1-2022/rtx5000-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
    elif 'A100' in s:
        xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/A100-Nov-1-2022/a100-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
    elif 'M4000' in s:
        print('xformers for M4000 hasn\'t been built yet.')
        # xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/A100-Nov-1-2022/a100-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
    else:
        print('GPU not matched to xformers binary so a one-size-fits-all binary was installed. If you have any issues, please build xformers using the Tools block below.')
        xformers_whl = download_release('https://raw.githubusercontent.com/Cyberes/xformers-compiled/main/various/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl')
    !pip install --force-reinstall "{xformers_whl}"

# Make sure your models storage directory exists
!mkdir -p "{model_storage_dir}/hypernetworks"
!mkdir -p "{repo_storage_dir}/stable-diffusion-webui/models/hypernetworks"

# Link the output folders to /notebooks/outputs
!mkdir -p "{repo_storage_dir}/stable-diffusion-webui/log/images"
!mkdir -p "{repo_storage_dir}/stable-diffusion-webui/outputs"
!ln -s "{repo_storage_dir}/stable-diffusion-webui/outputs" /notebooks/
!ln -s "{repo_storage_dir}/stable-diffusion-webui/log" "{repo_storage_dir}/stable-diffusion-webui/outputs"

!echo -e "\n===================================\nDone! If you're seeing this the process has exited successfully.\n"

## Clean up and restart the kernel

In [ ]:
%store -r symlink_to_notebooks model_storage_dir repo_storage_dir

# Get some storage back
!pip cache purge
!cd "{model_storage_dir}" && rm *.aria2
!apt remove --purge -y aria2 p7zip-full
!apt autoremove --purge -y
!apt clean

# Restart the kernel
import os
os.kill(os.getpid(), 9)

## Link the models directory

Create symlinks. The file will be stored in the models storage directory and linked to where the WebUI expects the files to be.

In [ ]:
import os
import glob

def delete_broken_symlinks(dir):
    deleted = False
    for file in os.listdir(dir):
        path = f'{dir}/{file}'
        if os.path.islink(path) and not os.path.exists(os.readlink(path)):
            print(f'Symlink broken, removing: {file}')
            os.unlink(path)
            deleted = True
    if deleted:
        print('')
    
def symlink_models(source_dir, filetype, destination_dir):
    # don't pass directory paths with trailing slash
    if not os.path.isdir(source_dir):
        print('Not a directory:', source_dir)
        return
    for file in os.listdir(source_dir):
        if file.endswith(filetype):
            path = f'{destination_dir}/{file}'
            if not os.path.exists(path):
                print(f'New model: {file}')
                !ln -s "{source_dir}/{file}" "{destination_dir}/{file}"
            !ls -la --block-size=GB "{destination_dir}/{file}"

# Check for broken symlinks and remove them
delete_broken_symlinks(f'{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion')
delete_broken_symlinks(f'{repo_storage_dir}/stable-diffusion-webui/models/hypernetworks')

# Link models
symlink_models(model_storage_dir, 'ckpt', f'{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion')

# Link models one level recursive
# You can organize your models in your model storage dir into sub-folders.
for file in os.listdir(model_storage_dir):
    if os.path.isdir(os.path.join(model_storage_dir, file)) and file != 'hypernetworks':
        symlink_models(os.path.join(model_storage_dir, file), 'ckpt', f'{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion')

# Link config yaml files
symlink_models(model_storage_dir, 'yaml', f'{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion')

# Link hypernetworks
symlink_models(f'{model_storage_dir}/hypernetworks', 'pt', f'{repo_storage_dir}/stable-diffusion-webui/models/hypernetworks')

In [ ]:
#Update Dreambooth & pip install requirements
!cd /notebooks/stable-diffusion-webui/extensions/sd_dreambooth_extension/
!git pull
!pip install -r requirements.txt

# Launch the WebUI

In [8]:
%store -r model_storage_dir repo_storage_dir activate_xformers activate_deepdanbooru activate_medvram disable_pickle_check gradio_port gradio_auth
%cd "{repo_storage_dir}/stable-diffusion-webui"

# Enable optional args automatically based on settings
x_arg = '--xformers' if activate_xformers else ''
dd_arg = '--deepdanbooru' if activate_deepdanbooru else ''
mvram_arg = '--medvram' if activate_medvram else ''
pickled = '--disable-safe-unpickle' if disable_pickle_check else ''
port = f'--port {gradio_port}' if gradio_port else '--share'
auth = f'--gradio-auth {gradio_auth} --enable-insecure-extension-access' if gradio_auth else '--enable-insecure-extension-access'

# Launch args go below:
!python webui.py {x_arg} {dd_arg} {mvram_arg} {pickled} {port} {auth} --gradio-debug